In [ ]:
!pip install sqlalchemy

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings("ignore")

from selenium import webdriver
from bs4 import BeautifulSoup


In [3]:
columns = ['title', 'category', 'content']
df = pd.DataFrame(columns=columns)

In [4]:
page_urls = []
f = open('data/urls.txt', 'r')
lines = f.readlines()
f.close()
for line in lines:
    page_urls.append(line.strip())
print(page_urls[:3])

['https://namu.wiki/w/%ED%94%BC%EC%95%84%EB%8B%88%EC%8A%A4%ED%8A%B8(%EB%A1%9C%EB%A7%8C%20%ED%8F%B4%EB%9E%80%EC%8A%A4%ED%82%A4%20%EC%98%81%ED%99%94)', 'https://namu.wiki/w/%EC%A0%9C1%EC%B0%A8%20%EC%84%B8%EA%B3%84%20%EB%8C%80%EC%A0%84', 'https://namu.wiki/w/%EA%B9%80%EC%98%81%EB%82%A8']


In [5]:
driver_path = "/usr/local/bin/chromedriver"

In [6]:
for page_url in page_urls:
    driver = webdriver.Chrome(driver_path)
    driver.get(page_url)
    req = driver.page_source
    driver.close()
    soup = BeautifulSoup(req, "html.parser")
    contents_table = soup.find(name="article")
    title = contents_table.find_all("h1")[0]
    
    if len(contents_table.find_all("ul")) > 0:
        category = contents_table.find_all("ul")[0]
    else:
        category = None
        
    content_paragraphs = contents_table.find_all(name="div", attrs={"class":"wiki-paragraph"})
    content_corpus_list = []
    
    if title is not None:
        row_title = title.text.replace("\n", " ")
    else:
        row_title = ""
    
    if content_paragraphs is not None:
        for paragraphs in content_paragraphs:
            if paragraphs is not None:
                content_corpus_list.append(paragraphs.text.replace("\n", " "))
            else:
                content_corpus_list.append("")
    else:
        content_corpus_list.append("")
    
    if category is not None:
        row_category = category.text.replace("\n", " ")
    else:
        row_category = ""
    
    row = [row_title, row_category, "".join(content_corpus_list)]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)

In [7]:
df.head()

,title,category,content
0,피아니스트(로만 폴란스키 영화),2002년 영화홀로코스트 영화실화 바탕 영화폴란드 영화프랑스 영화영국 영화독일 영화...,로만 폴란스키 감독 장편 연출 작품 [ 펼치기 · 접기 ] 피아니스트의 주요 수상...
1,제1차 세계 대전,제1차 세계 대전,제1차 세계 대전 관련 틀 [ 펼치기 · 접기 ]제1차 세계 대전 참전국 [ 펼치기...
2,김영남,최고인민회의 상임위원장북한 외무상조선로동당 국제비서조선로동당 6기 정치국 상무위원조...,은(는) 여기로 연결됩니다. 동명이인에 대한 내용은 김영남(동명이인) 문서를의...
3,휴스턴 애스트로스/2021년,휴스턴 애스트로스메이저 리그 베이스볼/2021년,상위 문서: 휴스턴 애스트로스/역사 휴스턴 애스트로스 2021시즌지구 순위경기...
4,2020 도쿄 올림픽/축구(남자)/B조,2020 도쿄 올림픽,상위 문서: 2020 도쿄 올림픽/축구(남자) 축구 축구 [ 펼치기 · 접기 ...


In [15]:
engine = create_engine('sqlite:///data.db', echo=False)

In [16]:
df.to_sql('data/data.db', con=engine, index_label='ID')